<a href="https://colab.research.google.com/github/trece37/akiles1337-colab/blob/main/ACHILLES_LSTM_V3_1_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# ACHILLES LSTM V3.1 - TRAINING PIPELINE
# Enciclopedia Akiles1337 - Proyecto LLM1337
# Fecha: 26 Noviembre 2025
# ============================================================

# CELDA 1: Setup Inicial y Verificacion de GPU
from google.colab import drive
import tensorflow as tf
import os

print('=' * 70)
print(' ACHILLES LSTM V3.1 - TRAINING PIPELINE')
print('=' * 70)

# Montar Google Drive
drive.mount('/content/drive')

# Verificar GPU
print(f"\nGPUs disponibles: {len(tf.config.list_physical_devices('GPU'))}")
print(f"TensorFlow version: {tf.__version__}")

if len(tf.config.list_physical_devices('GPU')) == 0:
    print('\n⚠️ ADVERTENCIA: No se detecto GPU')
    print('Entrenamiento sera LENTO (4-6 horas)')
else:
    gpu = tf.config.list_physical_devices('GPU')[0]
    print(f"\n✅ GPU detectada: {gpu.name}")
    print('Entrenamiento optimizado (45-90 minutos)')

 ACHILLES LSTM V3.1 - TRAINING PIPELINE
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

GPUs disponibles: 1
TensorFlow version: 2.19.0

✅ GPU detectada: /physical_device:GPU:0
Entrenamiento optimizado (45-90 minutos)


In [ ]:
# CELDA 2: Instalar dependencias e imports
!pip install -q pandas-ta scikit-learn

import numpy as np
import pandas as pd
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Attention, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
import matplotlib.pyplot as plt
import json
import pickle
from datetime import datetime

print('\n✅ Imports completados correctamente')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 136.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.

In [ ]:
# CELDA 3: Configuracion e Hiperparametros
# ============================================================
# RUTAS - AJUSTA SEGUN TU GOOGLE DRIVE
# ============================================================
DATAPATH = '/content/drive/MyDrive/AchillesTraining/data/XAUUSD.csv'
OUTPUTDIR = '/content/drive/MyDrive/AchillesTraining/output/v3.1'

# Hiperparametros segun Enciclopedia Akiles1337
WINDOW_SIZE = 60          # Ventana temporal: 60 minutos
TRAIN_TEST_SPLIT = 0.8    # 80% train, 20% test
EPOCHS = 100              # Maximo de epocas
BATCH_SIZE = 64           # Tamano del batch
LEARNING_RATE = 0.001     # Learning rate inicial
MODEL_VERSION = 'v3.1.0'  # Versionado semantico

# Crear carpetas de salida
os.makedirs(OUTPUTDIR, exist_ok=True)

print('=' * 70)
print(' CONFIGURACION')
print('=' * 70)
print(f' Ruta datos: {DATAPATH}')
print(f' Ruta salida: {OUTPUTDIR}')
print(f'\n Hiperparametros:')
print(f'   Window size: {WINDOW_SIZE} minutos')
print(f'   Train/Test: {TRAIN_TEST_SPLIT*100:.0f}% / {(1-TRAIN_TEST_SPLIT)*100:.0f}%')
print(f'   Epocas max: {EPOCHS}')
print(f'   Batch size: {BATCH_SIZE}')
print(f'   Learning rate: {LEARNING_RATE}')
print(f'   Version modelo: {MODEL_VERSION}')

 CONFIGURACION
 Ruta datos: /content/drive/MyDrive/AchillesTraining/data/XAUUSD.csv
 Ruta salida: /content/drive/MyDrive/AchillesTraining/output/v3.1

 Hiperparametros:
   Window size: 60 minutos
   Train/Test: 80% / 20%
   Epocas max: 100
   Batch size: 64
   Learning rate: 0.001
   Version modelo: v3.1.0


In [ ]:
# CELDA 4: Feature Engineering - 9 Indicadores Técnicos
# ============================================================
def add_technical_indicators(df):
    """
    Añade indicadores técnicos como features adicionales.
    Basado en Enciclopedia Akiles1337 - Capítulo 4

    Features:
        1. close      - Precio de cierre (base)
        2. rsi        - Relative Strength Index (sobrecompra/sobreventa)
        3. ema20      - EMA corto plazo
        4. ema50      - EMA medio plazo
        5. atr        - Average True Range (volatilidad)
        6. macd       - MACD
        7. macd_signal- Señal MACD
        8. bb_upper   - Bollinger Band Superior
        9. bb_lower   - Bollinger Band Inferior
    """
    print('Calculando indicadores técnicos...')

    # RSI - 14 periodos
    df['rsi'] = ta.rsi(df['close'], length=14)
    print('  RSI calculado (14 periodos)')

    # EMAs - 20 y 50 periodos
    df['ema20'] = ta.ema(df['close'], length=20)
    df['ema50'] = ta.ema(df['close'], length=50)
    print('  EMA calculadas (20, 50 periodos)')

    # ATR - volatilidad
    df['atr'] = ta.atr(df['high'], df['low'], df['close'], length=14)
    print('  ATR calculado (volatilidad)')

    # MACD + señal
    macd = ta.macd(df['close'])
    df['macd'] = macd['MACD_12_26_9']
    df['macd_signal'] = macd['MACDs_12_26_9']
    print('  MACD + señal calculado')

    # Bandas de Bollinger (robusto ante cambios de versión)
    bbands = ta.bbands(df['close'], length=20)
    print("  Columnas BBANDS generadas:", bbands.columns.tolist())    # Para verificar los nombres

    # Detecta los nombres automáticamente
    bb_upper_col = next(c for c in bbands.columns if "BBU" in c)
    bb_lower_col = next(c for c in bbands.columns if "BBL" in c)
    df['bb_upper'] = bbands[bb_upper_col]
    df['bb_lower'] = bbands[bb_lower_col]
    print('  Bollinger Bands calculadas')

    # Eliminar NaN generados por indicadores
    initial_rows = len(df)
    df = df.dropna()
    dropped_rows = initial_rows - len(df)
    print(f'\n  Filas eliminadas (NaN): {dropped_rows}')
    print(f'  Filas finales: {len(df)}')

    print('✅ Funcion add_technical_indicators definida')
    return df


In [ ]:
# CELDA 5: Cargar datos y preprocesar
# ============================================================
print('=' * 70)
print(' CARGANDO DATOS')
print('=' * 70)

# Cargar CSV
df = pd.read_csv(DATAPATH)
print(f'\n Archivo cargado: {DATAPATH}')
print(f' Filas totales: {len(df):,}')
print(f' Columnas: {list(df.columns)}')

# Mostrar primeras filas
print(f'\n Primeras filas:')
print(df.head())

# Aplicar feature engineering
print(f'\n' + '=' * 70)
print(' FEATURE ENGINEERING')
print('=' * 70)
df = add_technical_indicators(df)

# Seleccionar features para el modelo (9 features)
FEATURE_COLUMNS = ['close', 'rsi', 'ema20', 'ema50', 'atr', 'macd', 'macd_signal', 'bb_upper', 'bb_lower']
data = df[FEATURE_COLUMNS].values

print(f'\n Features seleccionadas: {FEATURE_COLUMNS}')
print(f' Shape datos: {data.shape}')

 CARGANDO DATOS

 Archivo cargado: /content/drive/MyDrive/AchillesTraining/data/XAUUSD.csv
 Filas totales: 50,000
 Columnas: ['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume']

 Primeras filas:
                  time     open     high      low    close  tick_volume  \
0  2025-10-07 10:19:00  3950.42  3950.71  3949.67  3950.00           85   
1  2025-10-07 10:20:00  3950.02  3950.47  3946.59  3947.76           98   
2  2025-10-07 10:21:00  3947.75  3948.47  3945.76  3947.45           97   
3  2025-10-07 10:22:00  3948.03  3948.03  3944.19  3944.19           94   
4  2025-10-07 10:23:00  3944.25  3947.78  3944.00  3947.32           91   

   spread  real_volume  
0      15            0  
1      15            0  
2       5            0  
3       6            0  
4       5            0  

 FEATURE ENGINEERING
Calculando indicadores técnicos...
  RSI calculado (14 periodos)
  EMA calculadas (20, 50 periodos)
  ATR calculado (volatilidad)
  MACD + señal calculad

In [ ]:
# CELDA 6: Train/Test Split y Normalizacion (SIN DATA LEAKAGE)
# ============================================================
# IMPORTANTE: Scaler se ajusta SOLO con datos de entrenamiento
# Segun ANEXO 2 de Enciclopedia Akiles1337
# ============================================================
print('=' * 70)
print(' TRAIN/TEST SPLIT Y NORMALIZACION')
print('=' * 70)

# Calcular punto de division
split_idx = int(len(data) * TRAIN_TEST_SPLIT)

# Dividir datos ANTES de normalizar
train_data = data[:split_idx]
test_data = data[split_idx:]

print(f'\n Train samples: {len(train_data):,}')
print(f' Test samples: {len(test_data):,}')

# Crear scaler y ajustar SOLO con datos de entrenamiento (evita data leakage)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_data)  # Solo fit con train!

# Transformar ambos conjuntos
train_scaled = scaler.transform(train_data)
test_scaled = scaler.transform(test_data)

print(f'\n✅ Scaler ajustado SOLO con datos de entrenamiento')
print(f' Rango normalizado: [0, 1]')
print(f' Train scaled shape: {train_scaled.shape}')
print(f' Test scaled shape: {test_scaled.shape}')

 TRAIN/TEST SPLIT Y NORMALIZACION

 Train samples: 39,960
 Test samples: 9,991

✅ Scaler ajustado SOLO con datos de entrenamiento
 Rango normalizado: [0, 1]
 Train scaled shape: (39960, 9)
 Test scaled shape: (9991, 9)


In [ ]:
# CELDA 7: Crear secuencias temporales para LSTM
# ============================================================
def create_sequences(data, window_size):
    """
    Crea secuencias temporales para entrenamiento LSTM.
    Input: ultimos 60 minutos (window_size)
    Output: precio del minuto siguiente (close)
    """
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])  # Ultimos 60 minutos
        y.append(data[i, 0])  # Precio close del siguiente minuto
    return np.array(X), np.array(y)

print('=' * 70)
print(' CREANDO SECUENCIAS TEMPORALES')
print('=' * 70)

# Crear secuencias para train y test
X_train, y_train = create_sequences(train_scaled, WINDOW_SIZE)
X_test, y_test = create_sequences(test_scaled, WINDOW_SIZE)

print(f'\n Secuencias creadas:')
print(f'   X_train shape: {X_train.shape} (samples, timesteps, features)')
print(f'   y_train shape: {y_train.shape}')
print(f'   X_test shape: {X_test.shape}')
print(f'   y_test shape: {y_test.shape}')

print(f'\n✅ Datos listos para entrenar Bi-LSTM')

 CREANDO SECUENCIAS TEMPORALES

 Secuencias creadas:
   X_train shape: (39900, 60, 9) (samples, timesteps, features)
   y_train shape: (39900,)
   X_test shape: (9931, 60, 9)
   y_test shape: (9931,)

✅ Datos listos para entrenar Bi-LSTM


In [ ]:
# CELDA 8: Arquitectura Bi-LSTM + Self-Attention (Achilles V3.1)
# ============================================================
# Segun Capitulo 3 de Enciclopedia Akiles1337
# ============================================================
def build_achilles_model(input_shape):
    """
    Arquitectura Achilles LSTM V3.1:
    - Bidirectional LSTM (64 neuronas) con L1/L2 regularization
    - Dropout 40%
    - BatchNormalization
    - Self-Attention Layer
    - Bidirectional LSTM (32 neuronas)
    - Dropout 30%
    - Dense output (1 neurona - precio predicho)
    """
    # Input Layer
    inputs = Input(shape=input_shape)

    # Bidirectional LSTM Layer 1 (64 neuronas x 2 direcciones)
    lstm1 = Bidirectional(
        LSTM(64, return_sequences=True,
             kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))
    )(inputs)
    lstm1 = Dropout(0.4)(lstm1)
    lstm1 = BatchNormalization()(lstm1)

    # Self-Attention Layer
    attention = Attention()([lstm1, lstm1])

    # Concatenate LSTM output con Attention
    concat = Concatenate()([lstm1, attention])

    # Bidirectional LSTM Layer 2 (32 neuronas x 2 direcciones)
    lstm2 = Bidirectional(
        LSTM(32, return_sequences=False)
    )(concat)
    lstm2 = Dropout(0.3)(lstm2)

    # Output Layer
    outputs = Dense(1)(lstm2)

    # Compilar modelo
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='mse',
        metrics=['mae']
    )

    return model

print('=' * 70)
print(' ARQUITECTURA ACHILLES LSTM V3.1')
print('=' * 70)

# Construir modelo
model = build_achilles_model((WINDOW_SIZE, len(FEATURE_COLUMNS)))
model.summary()

 ARQUITECTURA ACHILLES LSTM V3.1


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 9)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 60, 128)   │     37,888 │ input_layer[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 60, 128)   │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 60, 128)   │        512 │ dropout[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 60, 128)   │          0 │ batch_normalizat… │
│ (Attention)         │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 60, 256)   │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 64)        │     73,984 │ concatenate[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 112,449 (439.25 KB)

 Trainable params: 112,193 (438.25 KB)

 Non-trainable params: 256 (1.00 KB)

In [ ]:
# CELDA 9: Entrenamiento con Callbacks
# ============================================================
print('=' * 70)
print(' ENTRENAMIENTO')
print('=' * 70)

# Callbacks para optimizar entrenamiento
callbacks = [
    # Early Stopping - para si no mejora en 15 epocas
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    # Reducir LR si no mejora en 10 epocas
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        verbose=1
    ),
    # Guardar mejor modelo
    ModelCheckpoint(
        filepath=f'{OUTPUTDIR}/best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

print(f'\n Iniciando entrenamiento...')
print(f' Epocas max: {EPOCHS}')
print(f' Batch size: {BATCH_SIZE}')
print(f' Early stopping patience: 15')

# Entrenar modelo
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,  # 10% de train para validacion
    callbacks=callbacks,
    verbose=1
)

print(f'\n✅ Entrenamiento completado!')

 ENTRENAMIENTO

 Iniciando entrenamiento...
 Epocas max: 100
 Batch size: 64
 Early stopping patience: 15
Epoch 1/100
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0413 - mae: 0.1113
Epoch 1: val_loss improved from inf to 0.00521, saving model to /content/drive/MyDrive/AchillesTraining/output/v3.1/best_model.keras
562/562 ━━━━━━━━━━━━━━━━━━━━ 22s 27ms/step - loss: 0.0413 - mae: 0.1113 - val_loss: 0.0052 - val_mae: 0.0229 - learning_rate: 0.0010
Epoch 2/100
560/562 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0073 - mae: 0.0419
Epoch 2: val_loss improved from 0.00521 to 0.00381, saving model to /content/drive/MyDrive/AchillesTraining/output/v3.1/best_model.keras
562/562 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - loss: 0.0072 - mae: 0.0419 - val_loss: 0.0038 - val_mae: 0.0344 - learning_rate: 0.0010
Epoch 3/100
561/562 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0043 - mae: 0.0329
Epoch 3: val_loss improved from 0.00381 to 0.00191, saving model to /content/drive/MyDrive/AchillesTraining/o

In [ ]:
# CELDA 10: Evaluacion y Metricas
# ============================================================
print('=' * 70)
print(' EVALUACION EN TEST SET')
print('=' * 70)

# Predecir en test set
y_pred_scaled = model.predict(X_test)

# Desnormalizar predicciones y valores reales
# Crear array con ceros para las otras features
y_pred_padded = np.zeros((len(y_pred_scaled), len(FEATURE_COLUMNS)))
y_pred_padded[:, 0] = y_pred_scaled.flatten()
y_pred = scaler.inverse_transform(y_pred_padded)[:, 0]

y_test_padded = np.zeros((len(y_test), len(FEATURE_COLUMNS)))
y_test_padded[:, 0] = y_test
y_test_original = scaler.inverse_transform(y_test_padded)[:, 0]

# Calcular metricas
mae = mean_absolute_error(y_test_original, y_pred)
mape = mean_absolute_percentage_error(y_test_original, y_pred) * 100
r2 = r2_score(y_test_original, y_pred)

print(f'\n METRICAS FINALES:')
print(f'   MAE (Mean Absolute Error): ${mae:.2f}')
print(f'   MAPE (Mean Absolute % Error): {mape:.2f}%')
print(f'   R2 Score: {r2:.4f}')

# Comparar con objetivos de la enciclopedia
print(f'\n OBJETIVO ENCICLOPEDIA: MAE < $2.00')
if mae < 2.0:
    print(f'   ✅ OBJETIVO CUMPLIDO! MAE = ${mae:.2f}')
else:
    print(f'   ⚠️ Por mejorar. MAE = ${mae:.2f}')

 EVALUACION EN TEST SET
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

 METRICAS FINALES:
   MAE (Mean Absolute Error): $2.58
   MAPE (Mean Absolute % Error): 0.06%
   R2 Score: 0.9937

 OBJETIVO ENCICLOPEDIA: MAE < $2.00
   ⚠️ Por mejorar. MAE = $2.58


In [ ]:
# CELDA 11: Exportar Modelo Achilles para Vertex AI + Full Assets
# ===============================================================
print('=' * 70)
print(' EXPORTANDO MODELO ACHILLES Y ASSETS')
print('=' * 70)

# EXPORTAR modelo en formato TensorFlow SavedModel (Vertex AI Ready)
savedmodel_path = f'{OUTPUTDIR}/achilles_model_export'
model.export(savedmodel_path)  # Keras 3 exporta correcto a SavedModel como directorio
print(f'\n✅ SavedModel exportado a directorio: {savedmodel_path}')

# EXPORTAR scaler en formato pickle (para preprocessing en producción)
scaler_path = f'{OUTPUTDIR}/achilles_scaler_{MODEL_VERSION}.pkl'
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f'✅ Scaler guardado: {scaler_path}')

# EXPORTAR metadata del modelo en JSON (info para Vertex AI, Akiles y debugging futuro)
metadata = {
    'model_version': MODEL_VERSION,
    'training_date': datetime.now().isoformat(),
    'window_size': WINDOW_SIZE,
    'features': FEATURE_COLUMNS,
    'metrics': {
        'mae': float(mae),
        'mape': float(mape),
        'r2': float(r2)
    },
    'hyperparameters': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE
    }
}
metadata_path = f'{OUTPUTDIR}/model_metadata_{MODEL_VERSION}.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f'✅ Metadata guardada: {metadata_path}')

# RESUMEN FINAL (listado completo de assets para Vertex AI)
print(f'\n' + '=' * 70)
print(' ARCHIVOS GENERADOS')
print('=' * 70)
print(f' 1. {savedmodel_path}/')         # SavedModel DIR
print(f' 2. {scaler_path}')              # Scaler PKL
print(f' 3. {metadata_path}')            # Metadata JSON
print(f'\n🚀 Modelo listo para desplegar en Vertex AI!')
print(f'\n Siguiente paso: Subir a gs://llm1337-trading-data/models/')


 EXPORTANDO MODELO ACHILLES Y ASSETS
Saved artifact at '/content/drive/MyDrive/AchillesTraining/output/v3.1/achilles_model_export'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132680547996240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680548001616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680547999696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680547997008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680567278288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680542821008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680542822928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680542824272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132680542824080: TensorSpec(shape=(), dtype=tf.resource, name=None

In [ ]:
!cp "/content/drive/MyDrive/AchillesTraining/ACHILLES-LSTM-V3.1-Training.ipynb" "/content/"
!jupyter nbconvert --to html "/content/ACHILLES-LSTM-V3.1-Training.ipynb"
# El HTML aparecerá en /content/ACHILLES-LSTM-V3.1-Training.html
# Luego lo puedes mover a Drive si lo quieres:
!mv "/content/ACHILLES-LSTM-V3.1-Training.html" "/content/drive/MyDrive/AchillesTraining/"


cp: cannot stat '/content/drive/MyDrive/AchillesTraining/ACHILLES-LSTM-V3.1-Training.ipynb': No such file or directory
[NbConvertApp] WARNING | pattern '/content/ACHILLES-LSTM-V3.1-Training.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterAp